In [1]:
import mnist_loader

In [2]:
training_data, validation_data, test_data = mnist_loader.load_data_wrapper()

In [4]:
import numpy as np
import random

class Network(object):

    def __init__(self, sizes: list[int]):
        """sizes contains the number of neurons in the respective layers

        to create a NN with 3 layers, where layer 0 has size=2, layer 1 has size=3, final layer has size=1
        wed init net = Network([2, 3, 1])
        """
        self.num_layers = len(sizes)
        self.sizes = sizes
        self.biases = [np.random.randn(y, 1) for y in sizes[1:]]
        self.weights = [np.random.randn(y, x) for x, y in zip(sizes[:-1], sizes[1:])]

    def sigmoid(self, z: float) -> float:
        return 1.0 / (1.0 + np.exp(-z))

    # def feedforward(self, a: np.ndarray) -> np.ndarray:
    #     """a' = sig(wa+b)"""
    #     for layer_index in range(self.num_layers - 1):
    #         a = self.sigmoid(
    #             np.dot(self.weights[layer_index], a) + self.biases[layer_index]
    #         )

    #     print(f"Result of feedforward:\n{a}")
    #     return a

    def feedforward(self, a):
        """Return the output of the network if "a" is input."""
        for b, w in zip(self.biases, self.weights):
            a = self.sigmoid(np.dot(w, a)+b)
        return a

    def SGD(self, training_data, epochs, mini_batch_size, eta, test_data=None):
        """Train the neural network using mini-batch stochastic
        gradient descent.  The "training_data" is a list of tuples
        "(x, y)" representing the training inputs and the desired
        outputs.  The other non-optional parameters are
        self-explanatory.  If "test_data" is provided then the
        network will be evaluated against the test data after each
        epoch, and partial progress printed out.  This is useful for
        tracking progress, but slows things down substantially."""
        if test_data:
            print(type(test_data))
            n_test = len(test_data)
        n = len(training_data)
        for j in np.arange(epochs):
            random.shuffle(training_data)
            mini_batches = [
                training_data[k : k + mini_batch_size]
                for k in np.arange(0, n, mini_batch_size)
            ]
            for mini_batch in mini_batches:
                self.update_mini_batch(mini_batch, eta)
            if test_data:
                print(f"Epoch {j}: {self.evaluate(test_data)} {n_test}")
                
            else:
                print(f"Epoch {j} complete")

    def update_mini_batch(self, mini_batch, eta):
        """Update the network's weights and biases by applying
        gradient descent using backpropagation to a single mini batch.
        The "mini_batch" is a list of tuples "(x, y)", and "eta"
        is the learning rate."""
        nabla_b = [np.zeros(b.shape) for b in self.biases]
        nabla_w = [np.zeros(w.shape) for w in self.weights]
        for x, y in mini_batch:
            delta_nabla_b, delta_nabla_w = self.backprop(x, y)
            nabla_b = [nb + dnb for nb, dnb in zip(nabla_b, delta_nabla_b)]
            nabla_w = [nw + dnw for nw, dnw in zip(nabla_w, delta_nabla_w)]
        self.weights = [
            w - (eta / len(mini_batch)) * nw for w, nw in zip(self.weights, nabla_w)
        ]
        self.biases = [
            b - (eta / len(mini_batch)) * nb for b, nb in zip(self.biases, nabla_b)
        ]

    def backprop(self, x, y):
        """Return a tuple ``(nabla_b, nabla_w)`` representing the
        gradient for the cost function C_x.  ``nabla_b`` and
        ``nabla_w`` are layer-by-layer lists of numpy arrays, similar
        to ``self.biases`` and ``self.weights``."""
        nabla_b = [np.zeros(b.shape) for b in self.biases]
        nabla_w = [np.zeros(w.shape) for w in self.weights]
        # feedforward
        activation = x
        activations = [x]  # list to store all the activations, layer by layer
        zs = []  # list to store all the z vectors, layer by layer
        for b, w in zip(self.biases, self.weights):
            z = np.dot(w, activation) + b
            zs.append(z)
            activation = self.sigmoid(z)
            activations.append(activation)
        # backward pass
        delta = self.cost_derivative(activations[-1], y) * self.sigmoid_prime(zs[-1])
        nabla_b[-1] = delta
        nabla_w[-1] = np.dot(delta, activations[-2].transpose())
        """Note that the variable l in the loop below is used a little
        differently to the notation in Chapter 2 of the book.  Here,
        l = 1 means the last layer of neurons, l = 2 is the
        second-last layer, and so on.  It's a renumbering of the
        scheme in the book, used here to take advantage of the fact
        that Python can use negative indices in lists."""
        for l in np.arange(2, self.num_layers):
            z = zs[-l]
            sp = self.sigmoid_prime(z)
            delta = np.dot(self.weights[-l + 1].transpose(), delta) * sp
            nabla_b[-l] = delta
            nabla_w[-l] = np.dot(delta, activations[-l - 1].transpose())
        return (nabla_b, nabla_w)

    def evaluate(self, test_data):
        """Return the number of test inputs for which the neural
        network outputs the correct result. Note that the neural
        network's output is assumed to be the index of whichever
        neuron in the final layer has the highest activation."""
        test_results = [(np.argmax(self.feedforward(x)), y) for (x, y) in test_data]
        return sum(int(x == y) for (x, y) in test_results)

    def cost_derivative(self, output_activations, y):
        """Return the vector of partial derivatives partial C_x partial a for the output activations."""
        return output_activations - y
    
    def sigmoid_prime(self, z):
        """Derivative of the sigmoid function."""
        return self.sigmoid(z)*(1-self.sigmoid(z))

def main():
    size1 = [2, 3, 1]
    net = Network(size1)
    input = np.array([0.5, 1])
    net.feedforward(input)


net = Network([784, 30, 10])
net.SGD(training_data, 30, 10, 3.0, test_data=test_data)

<class 'list'>
Epoch 0: 8988 10000
Epoch 1: 9169 10000
Epoch 2: 9245 10000
Epoch 3: 9326 10000
Epoch 4: 9366 10000
Epoch 5: 9361 10000
Epoch 6: 9381 10000
Epoch 7: 9425 10000
Epoch 8: 9456 10000
Epoch 9: 9425 10000
Epoch 10: 9462 10000
Epoch 11: 9421 10000
Epoch 12: 9424 10000
Epoch 13: 9453 10000
Epoch 14: 9426 10000
Epoch 15: 9449 10000
Epoch 16: 9459 10000
Epoch 17: 9465 10000
Epoch 18: 9479 10000
Epoch 19: 9476 10000
Epoch 20: 9457 10000
Epoch 21: 9478 10000
Epoch 22: 9468 10000
Epoch 23: 9479 10000
Epoch 24: 9458 10000
Epoch 25: 9473 10000
Epoch 26: 9464 10000
Epoch 27: 9463 10000
Epoch 28: 9492 10000
Epoch 29: 9475 10000


In [5]:
net = Network([784, 30, 30, 10])
net.SGD(training_data, 30, 10, 3.0, test_data=test_data)

<class 'list'>
Epoch 0: 8975 10000
Epoch 1: 9150 10000
Epoch 2: 9290 10000
Epoch 3: 9351 10000
Epoch 4: 9310 10000
Epoch 5: 9366 10000
Epoch 6: 9409 10000
Epoch 7: 9424 10000
Epoch 8: 9430 10000
Epoch 9: 9426 10000
Epoch 10: 9440 10000
Epoch 11: 9471 10000
Epoch 12: 9432 10000
Epoch 13: 9430 10000
Epoch 14: 9449 10000
Epoch 15: 9450 10000
Epoch 16: 9407 10000
Epoch 17: 9482 10000
Epoch 18: 9388 10000
Epoch 19: 9479 10000
Epoch 20: 9489 10000
Epoch 21: 9467 10000
Epoch 22: 9469 10000
Epoch 23: 9487 10000
Epoch 24: 9450 10000
Epoch 25: 9462 10000
Epoch 26: 9467 10000
Epoch 27: 9471 10000
Epoch 28: 9457 10000
Epoch 29: 9497 10000


In [6]:
net = Network([784, 100, 10])
net.SGD(training_data, 30, 10, 3.0, test_data=test_data)

<class 'list'>
Epoch 0: 7589 10000
Epoch 1: 7770 10000
Epoch 2: 8490 10000
Epoch 3: 9383 10000
Epoch 4: 9450 10000
Epoch 5: 9474 10000
Epoch 6: 9533 10000
Epoch 7: 9557 10000
Epoch 8: 9556 10000
Epoch 9: 9561 10000
Epoch 10: 9580 10000
Epoch 11: 9589 10000
Epoch 12: 9601 10000
Epoch 13: 9584 10000
Epoch 14: 9621 10000
Epoch 15: 9611 10000
Epoch 16: 9627 10000
Epoch 17: 9614 10000
Epoch 18: 9621 10000
Epoch 19: 9635 10000
Epoch 20: 9624 10000
Epoch 21: 9646 10000
Epoch 22: 9649 10000
Epoch 23: 9651 10000
Epoch 24: 9648 10000
Epoch 25: 9661 10000
Epoch 26: 9645 10000
Epoch 27: 9645 10000
Epoch 28: 9642 10000
Epoch 29: 9642 10000


In [ ]:
net = Network([784, 30,30, 30, 30, 30, 30, 30 , 10])
net.SGD(training_data, 30, 10, 3.0, test_data=test_data)